In [ ]:
# Dans votre notebook (01_strategy_dev.ipynb)

# Important : Si votre 'src' est au même niveau que 'notebooks',
# cela permet d'importer les modules depuis le dossier src/
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import MetaTrader5 as mt5
from src.data_fetcher import initialize_mt5, getDataFromMT5, shutdown_mt5
from src.strategy import add_moving_average_signals

# --- 1. Connexion et Récupération des Données ---
if initialize_mt5():
    
    SYMBOL = "EURUSD"
    TIMEFRAME = mt5.TIMEFRAME_H1 # Timeframe de 1 heure
    N_BARS = 1000                # Récupérer 1000 barres

    # LIGNE CORRIGÉE (get_data_from_mt5 en minuscules)
    df_raw = getDataFromMT5(SYMBOL, TIMEFRAME, N_BARS)

    # N'oubliez pas de fermer la connexion
    # (Déplacé ici pour que la connexion se ferme même si les données sont None)
    shutdown_mt5() 

    if df_raw is not None:
        print("\n--- Données Brutes (dernières 5 lignes) ---")
        print(df_raw.tail())

        # --- 2. Calcul de la Stratégie ---
        df_strategy = add_moving_average_signals(df_raw, short_window=50, long_window=200)
        
        print("\n--- Données avec Stratégie (dernières 10 lignes) ---")
        print(df_strategy.tail(10))
        
        # Filtrer pour ne voir que les signaux
        signals = df_strategy[df_strategy['signal'] != 0]
        print("\n--- Derniers signaux générés ---")
        print(signals.tail())

else:
    print("Vérifiez que votre terminal MetaTrader 5 est bien lancé.")
    # Si l'initialisation échoue, on ferme quand même au cas où
    shutdown_mt5()


MT5 initialized successfully
Connexion à MetaTrader 5 fermée.


In [15]:
import pandas as pd
import numpy as np

def run_simple_backtest(df_strategy, initial_cash=10000):
    print("Lancement du backtest simple (sans vectorbt)...")

    position = df_strategy['signal'].replace(0, np.nan).ffill().fillna(0)

    position[position == -1] = 0

    market_return = df_strategy['close'].pct_change()

    strategy_return = market_return * position.shift(1)


    cumulative_market = (1 + market_return).cumprod()
    cumulative_strategy = (1 + strategy_return.fillna(0)).cumprod()
    

    total_trades = (position.diff() == 1).sum()
    final_value = cumulative_strategy.iloc[-1] * initial_cash
    total_return_pct = (final_value - initial_cash) / initial_cash * 100
    
    print("\n--- Statistiques du Backtest Simple ---")
    print(f"Capital Initial: {initial_cash:,.2f}")
    print(f"Valeur Finale: {final_value:,.2f}")
    print(f"Retour Total: {total_return_pct:.2f}%")
    print(f"Nombre de trades (long only): {total_trades}")

    results = pd.DataFrame({
        'Buy_and_Hold': cumulative_market,
        'Strategy': cumulative_strategy
    })
    
    return results

# Exécuter le backtest simple

run_simple_backtest(df_strategy)

print("Module de stratégie chargé.")

NameError: name 'df_strategy' is not defined